<a href="https://colab.research.google.com/github/qianqiancui/Pisturk-Trait-Learning-Task/blob/master/notebooks/fairface_VGG_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Race prediction from face images using Pytorch



* Gender : `Male` or `Female`
* Race : based on the labels in the datasets


## 1. Prepare


Before you run the note book, click **runtime**, select **change runtime type**, choose **GPU**. Then click **run all** or run sections manually based on your need.

I strongly suggest that you create a copy of the notebook each time you manipulate the # of images for black and white faces or make big changes.

### load resources

In [ ]:
# link google drive to colab, so that we are able to load drive files into the colab notebook
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# load repository to colab "dl_repo" is the name of the repo
! pip install /content/gdrive/My\ Drive/dl_repo

# import python modules 
import torch
import numpy as np
import tqdm
import glob
import os

# import our own modules included in the scripts
import torchvision.transforms as transforms
from vision_utils.custom_torch_utils import load_model
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model, PretrainedMT
from multitask_rag.train import run_utk
from multitask_rag.utk_data_utils import get_utk_dataloader, split_utk
from multitask_rag.evaluate import evaluate_model as eval_utk

Processing ./gdrive/My Drive/dl_repo
     |████████████████████████████████| 184kB 4.5MB/s 
     |████████████████████████████████| 317kB 14.0MB/s 
  Created wheel for democlassi: filename=democlassi-0.5-cp36-none-any.whl size=10069745 sha256=7f6d98b7c308896d40eccbaf1e5fd6842050f1b9b733abd6c0ff68d12144b949
  Stored in directory: /tmp/pip-ephem-wheel-cache-95hp8tvk/wheels/3b/65/f2/004ee8c3b812c6b226984d1dadd6248e7732b15ea4422ca91d
Successfully built democlassi


In [ ]:
# Create a directory where to store data
os.makedirs('/content/data', exist_ok=True)

# load the dataset and unzip 
!cp /content/gdrive/My\ Drive/ff.zip /content/data/
!unzip data/ff.zip -d data/

In [ ]:
# print content of the data directory
!ls /content/data/

# check # of images
list_images = glob.glob('/content/data/ff/*jp*')
print('total images:',len(list_images))

We have 86744+10954 images in total. The image names format is the following : `age_gender_race_date`.
for instance this image name `1_0_0_20161219140623097.jpg.chip.jpg` suggests: age is `1`, gender is `0` (Male) and race is `0` (White).
However there are few images for which the name is malfomed, so we remove them using the following code snippet :

In [ ]:
# function to remove invalid images (that he filenames is not correctly formatted)
def get_invalid_images(root_path='/content/data/ff/'):
    list_files = glob.glob(os.path.join(root_path, '*.[jJ][pP]*'))
    filenames = [path.split('/')[-1].split('_') for path in list_files]
    invalid_images = []
    for i, im in enumerate(tqdm.tqdm(filenames)):
      # check if the 1st, 2nd, 3rd parts of a given file name are digits
      # As a recap, image name goes like 1_1_1_00000, 
# which means age_gender_race_unrelated info
        if im[0].isdigit() and im[1].isdigit() and im[2].isdigit():
            continue
        else:
            invalid_images.append(list_files[i])
    return invalid_images

In [ ]:
# check # of invalid images
invalid_images = get_invalid_images()
print(invalid_images)

### remove unwanted races & manipulate the # of images for each race

In [ ]:
# function to remove unwanted races (in our case, only include black & white faces)
# change path based on your needs
def get_bw_images(root_path='/content/data/ff/'):
    list_files = glob.glob(os.path.join(root_path, '*.[jJ][pP]*'))
    filenames = [path.split('/')[-1].split('_') for path in list_files]
    invalid_races = []
    #find images based on its names
    for i, im in enumerate(tqdm.tqdm(filenames)):
      #only include 0&1, which means black & white
      # As a recap, image name goes like 1_1_1_00000, 
# which means age_gender_race_unrelated info
        if im[2] == '0' or im[2] == '1':
            continue
        else:
            invalid_races.append(list_files[i])

    return invalid_races

In [ ]:
# check # of faces from unwanted races
invalid_races = get_bw_images()
print(len(invalid_races))

In [ ]:
# Remove invalid files
for f in invalid_races:
    os.remove(f)

In [ ]:
# change # of faces for black & white faces based on our needs
black_list = []
white_list = []
# change based on your needs
def get_ratio(root_path='/content/data/ff/'):
    list_files = glob.glob('/content/data/ff/*jp*')
#find images based on its names
    filenames = [path.split('/')[-1].split('_') for path in list_files]
    invalid_white = []
    invalid_black = []
    for t, tm in enumerate(tqdm.tqdm(filenames)):
#check if the image is black
# As a recap, image name goes like 1_1_1_00000, 
# which means age_gender_race_unrelated info
      if tm[2] =='1':
        black_list.append(list_files[t])
        #stop adding black faces when reaching 10000. Change the number based on your need
        if len(black_list)>10000:
          # add unwanted blacks to one list
          invalid_black = black_list[10000:] 
  # same goes for the white faces
      elif tm[2] =='0':
        white_list.append(list_files[t])
        if len(white_list)>5000:
          invalid_white = white_list[5000:]

# record unwanted faces
    return invalid_black, invalid_white 


In [ ]:
invalid_black, invalid_white = get_ratio()
# check the # of unwanted faces for each race
print(len(invalid_white))
print(len(invalid_black))

In [ ]:
# remove unwanted faces from the set
for f in invalid_white:
    os.remove(f)

for f in invalid_black:
    os.remove(f)

### double check 

In [ ]:
# double check # of faces for each races 

def display_examples_utk(root_path, label_type, label_value):
# add labels based on the digit of the file name. As a recap, image name goes like 1_1_1_00000, 
# which means age_gender_race_unrelated info
    list_files = [item for item in glob.glob(os.path.join(root_path, '*.jpg'))
                  if len(item.split('/')[-1].split('_')[:-1]) == 3]

    labels = [item.split('/')[-1].split('_')[:-1] for item in list_files]
# create labels
    labels = {
        'age': [int(item[0]) for item in labels],
        'gender': [int(item[1]) for item in labels],
        'race': [int(item[2]) for item in labels]
    }
# name the labels
    label_names = {
        "race": {0: 'White', 1: 'Black', 2: 'East Asian', 3: 'Southeast Asian', 4: 'Middle Eastern', 5: 'Latino_Hispanic', 6:'Indian'},
        "gender": {0: 'Male', 1: 'Female'},
    }

# list the # of faces for each race
    print('Number of images for {} : {}'.format(
        label_type, label_names[label_type][label_value] if label_type != 'age' else label_value
    ))
    inds = [ind for ind, lab in enumerate(labels[label_type]) if lab == label_value]

    print(len(inds))

if __name__ == '__main__':
  display_examples_utk('/content/data/ff/', 'race', 0)
  display_examples_utk('/content/data/ff/', 'race', 1)
  display_examples_utk('/content/data/ff/', 'race', 2)
  display_examples_utk('/content/data/ff/', 'race', 3)
  display_examples_utk('/content/data/ff/', 'race', 4)
  display_examples_utk('/content/data/ff/', 'race', 5)
  display_examples_utk('/content/data/ff/', 'race', 6)

Number of images for race : White
5000
Number of images for race : Black
10000
Number of images for race : East Asian
0
Number of images for race : Southeast Asian
0
Number of images for race : Middle Eastern
0
Number of images for race : Latino_Hispanic
0
Number of images for race : Indian
0


## 2. Some data pre-processing operations

Now that we have our dataset ready let's split it into training (70%), test (15%) and validation (15%) sets:

In [ ]:
# split the dataset into train, test and validation sets 
SRC_DIR = '/content/data/ff/'  # path to the folder containing all images
DEST_DIR = '/content/data/ff_split/' # path where to save the split dataset, 3 subdirectories will be created (train, valid and test)
SPLIT = 0.7 # ratio of the train set, the remaining (30%) will be split equally between validation and test sets

In [ ]:
split_utk(SRC_DIR, DEST_DIR, SPLIT)

  9%|▉         | 921/10499 [00:00<00:01, 9205.29it/s]

------------Copying train images-------------


 19%|█▉        | 433/2250 [00:00<00:00, 4319.36it/s]

------------Copying valid images-------------


 35%|███▌      | 794/2251 [00:00<00:00, 7648.32it/s]

------------Copying test images-------------


100%|██████████| 2251/2251 [00:00<00:00, 4808.85it/s]


Let's also define some transformations we would like to apply :
* Resize all images to 128 x 128
* convert them to pytorch tensors before feeding to the model

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

The last step before training is to create  DataLoader objects, which are pytorch generator-like objects for yielding data into batches during training :

In [ ]:
train_loader = get_utk_dataloader(batch_size=128, data_dir=DEST_DIR, data_transforms=data_transforms, flag='train')
val_loader = get_utk_dataloader(batch_size=128, data_dir=DEST_DIR, data_transforms=data_transforms, flag='valid')

my_data_loaders = {
    'train': train_loader,
    'valid': val_loader
}

## 3. Training

We'll trry three different architectures : 
* CNN based on Depthwise separable convolutions blocks
* Finetuning a pretrained Resnet50
* Finetuning a pretrained VGG19

### 3.1 Deptwhise Separable Convolutions architecture

In [ ]:
# Sepconv with adam

my_model = SepConvModelMT(dropout=0.7, n_class=[1, 2, 7], n_filters=[64, 128, 256, 512], kernels_size=[3, 3, 3, 3])
my_optimizer = torch.optim.Adam(my_model.parameters(), lr=1e-3)


We have created above a model and an Adam optimizer. The model has the following parameters :
* `dropout` = 0.7 : we apply a droput of 70%
* `n_class` = [1, 2, 6] : the model has 3 outputs that are age (a scalar thus a shape of 1), gender (male or female thus a shape of 2) and race (white, black, asian, indian or other thus a shape of 6).
*`n_filters` =  [64, 128, 256, 512] : number of features maps for each conv block
* `kernels_size` = [3, 3, 3, 3] : conv kernel size for each conv block

I also need to periodically backup the model's checkpoints in my google drive in case I get disconnected to google colab due to connectivity issues or anything else :


In [ ]:
backup_path = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/sep_conv_adam"
os.makedirs(backup_path, exist_ok=True)

Before starting training, let's clarify some important input arguments to the training function that are not quite clear :
* `log_interval` : print the training loss each `log_interval` iterations
* `dirname` : path to the directory where to locally save the best model checkpoint after each epoch
* `filename_prefix` : file name under which to save the model checkpoint
* `n_saved` : save the `n_saved` best models
* `launch_tensorboard` and `log_dir` :  whether to write tensorboard summaries, and if True, write them under `log_dir` folder
* `patience`: number of epochs to wait for before stopping training if no improvement is observed
* `resume_model` and `resume_optimizer` : optional paths to previously trained model and optimizer to start use them as starting point for the training
* `backup_step` and `backup_path` : copy the saved checkpoints from `dirname` to `backup_path` each `backup_step` epochs
* `n_epochs_freeze` : unfreeze the frozen layers after `n_epochs_freeze`, this is particularly used in case of finetuning a pretrained model
* `lr_after_freeze` : in case of finetuning a pretrained model, new learning rate to set after unfreezing frozen layers
* `loss_weights` : the model's outputs (age, gender, race) are not of the same magnitude, wo we made need to assign them different weights representing their respective contributions to the global loss



Now that it's a little bit clearer let's start training :

In [ ]:
run_utk(my_model, my_optimizer, epochs=300, log_interval=1, dataloaders=my_data_loaders,
        dirname='/content/checkpoints/sep_conv_adam', filename_prefix='sep_conv_adam', n_saved=1,
        log_dir=None, launch_tensorboard=False, patience=50,
        resume_model=None, resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=0, lr_after_freeze=None,
        loss_weights=[1/10, 1/0.16, 1/0.44])

### 3.2 Finetunig a pretrained Resnet50

We are going to :
* freeze all other layers and only train the classification using a learning rate of 1e-3
* unfreeze all layers after 10 epochs and finetune the whole model using a learning rate of 1e-4

In [ ]:
backup_path = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam"
os.makedirs(backup_path, exist_ok=True)

In [ ]:
num_classes=3
my_model = PretrainedMT(model_name='resnet', feature_extract=True, use_pretrained=True)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3)

In [ ]:
import sys
sys.path.append('../../vision_utils')
import torch.optim as optim
from vision_utils.custom_torch_utils import initialize_model
import torch.nn as nn


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


my_model = ConvModelMultiTask()
# Define the optimizer
optimizer = optim.Adam(
    [
        {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
        {"params": my_model.output_age.parameters(), "lr": 1e-3},
        {"params": my_model.output_gender.parameters(), "lr": 1e-3},
        {"params": my_model.output_race.parameters(), "lr": 1e-3},
        {"params": my_model.conv_base.conv1.parameters()},
        {"params": my_model.conv_base.layer1.parameters()},
        {"params": my_model.conv_base.layer2.parameters()},
        {"params": my_model.conv_base.layer3.parameters()},
        {"params": my_model.conv_base.layer4.parameters()},
    ],
    lr=1e-6,
)

In [ ]:
class PretrainedMT(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race

In [ ]:
run_utk(my_model, my_optimizer, epochs=300, log_interval=1, dataloaders=my_data_loaders,
        dirname='/content/checkpoints/resnet_adam', filename_prefix='resnet', n_saved=1,
        log_dir='/content/logs', launch_tensorboard=False, patience=50,
        resume_model=None, resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=10, n_cycle=None, lr_after_freeze=1e-4,
        loss_weights=[1/10, 1/0.16, 1/0.44], lr_plot=True)


### 3.3 Finetuning a pretrained VGG19

In [ ]:
backup_path = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/vgg_adam"
os.makedirs(backup_path, exist_ok=True)

In [ ]:
my_model = PretrainedMT(model_name='vgg', feature_extract=True, use_pretrained=True)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3)

In [ ]:
run_utk(my_model, my_optimizer, epochs=300, log_interval=1, dataloaders=my_data_loaders,
        dirname='/content/checkpoints/vgg_adam', filename_prefix='vgg', n_saved=1,
        log_dir='/content/logs', launch_tensorboard=False, patience=50,
        resume_model=None, resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=10, n_cycle=None, lr_after_freeze=1e-4,
        loss_weights=[1/10, 1/0.16, 1/0.44], lr_plot=True)

## 4. Evaluation

Now that our 3 models are trained we can evaluate them on the test set

In [ ]:
# check the checkpoints stored in the directory
!ls "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/cam/vgg_adam/white"
# !ls "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/sep_conv_adam"
# !ls "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/vgg_adam"

In [ ]:
# create test data loader
test_loader = get_utk_dataloader(batch_size=256, data_dir=DEST_DIR, data_transforms=data_transforms, flag='test')

In [ ]:
# Load the three models

# paths to the saved models
# path_sep_conv = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/sep_conv_adam/sep_conv_adam_checkpoint_val_loss=-4.26717553605379.pth"
path_resnet = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam/resnet_checkpoint_val_loss=-4.02144347319082.pth"
# path_vgg = "/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/vgg_adam/vgg_checkpoint_val_loss=-4.070847482863642.pth"

cpu_or_gpu = "cuda" if torch.cuda.is_available() else "cpu"
# sep_conv_model = SepConvModelMT()
# sep_conv_model.load_state_dict(torch.load(path_sep_conv, map_location=cpu_or_gpu)['model'])

import sys
sys.path.append('../../vision_utils')
import torch.optim as optim
from vision_utils.custom_torch_utils import initialize_model
import torch.nn as nn


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


my_model = ConvModelMultiTask()
# Define the optimizer
optimizer = optim.Adam(
    [
        {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
        {"params": my_model.output_age.parameters(), "lr": 1e-3},
        {"params": my_model.output_gender.parameters(), "lr": 1e-3},
        {"params": my_model.output_race.parameters(), "lr": 1e-3},
        {"params": my_model.conv_base.conv1.parameters()},
        {"params": my_model.conv_base.layer1.parameters()},
        {"params": my_model.conv_base.layer2.parameters()},
        {"params": my_model.conv_base.layer3.parameters()},
        {"params": my_model.conv_base.layer4.parameters()},
    ],
    lr=1e-6,
)

class PretrainedMT(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race

resnet_model = PretrainedMT(model_name='resnet')
resnet_model.load_state_dict(torch.load(path_resnet, map_location=cpu_or_gpu)['model'])

### Evaluate model: 

In [ ]:
# Import
import torch
import torch.nn.functional as F
import tqdm
import numpy as np
from vision_utils.custom_torch_utils import plot_confusion_matrix
from vision_utils.custom_torch_utils import processing_time
from torchvision import transforms
import time

def processing_time(func):
    """
    utility function to print execution time of a given function

    :param func: a python function to track the execution time
    :return:
    """
    def func_wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        seconds = time.time() - start
        m, s = divmod(seconds, 60)
        h, m = divmod(m, 60)
        print(f"The execution took {h} hours | {m} minutes | {s:.1f} seconds!")
    return func_wrapper


@processing_time
def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix

        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)




### Evaluate separable convolution model : 

In [ ]:
evaluate_model(sep_conv_model, test_loader, title='Evaluation on test set - Separable conv model')

### Evaluate resnet model

In [ ]:
evaluate_model(resnet_model, test_loader, title='Evaluation on test set - Resnet conv model')

### Evaluate VGG model

In [ ]:
evaluate_model(vgg_model, test_loader, title='Evaluation on test set - VGG model')

### normal-male

In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps



# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])






    for v in range(11):
      image_list = []
      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mor_male/m'+ str(v+1)
      # image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/M/morphed_'+ str(v+1)
      # image_id = '/Users/a3055/Desktop/mor_male/m' + str(v + 1)

      for i in range(11):
          if i < 10:
              images = image_id+ '_0'+ str(i) + '.jpg'
          else:
              images = image_id+ '_10.jpg'

          
          source_file = images


          frame = cv2.imread(source_file)
          frame = predict_from_frame(net, frame, model_utk, display_probs = True)

          parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name

          cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m_res/', f_name+'_predicted.jpg'), frame)
          cv2_imshow(frame)
          key = cv2.waitKey(0) & 0x00
          if key == ord("q"):
            cv2.destroyAllWindows()



###normal-female


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            # do age, gender and race prediction
            # face = frame[startY-25: endY+25, startX-25: endX+25]
            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)
    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])

    for v in range(11):
      image_list = []
      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mor_female/F'+ str(v+1)


      for i in range(11):
          if i < 10:
              images = image_id+ '_0'+ str(i) + '.jpg'
          else:
              images = image_id+ '_10.jpg'

          image_list.append(images)         
          source_file = images



          frame = cv2.imread(source_file)
          frame = predict_from_frame(net, frame, model_utk, display_probs = True)

          parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name

          cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f_res/', f_name+'_predicted.jpg'), frame)
          cv2_imshow(frame)
          # cv2_imshow('Face Detector', frame)
          key = cv2.waitKey(0) & 0x00
          if key == ord("q"):
            cv2.destroyAllWindows()



###SINGLE IMAGE TEST


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam/resnet_checkpoint_val_loss=-4.02144347319082.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            # do age, gender and race prediction
            # face = frame[startY-25: endY+25, startX-25: endX+25]
            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])



    source_file = "/content/gdrive/My Drive/DeepLearning/Face_detection/test/test25.jpg"
    frame = cv2.imread(source_file)

    frame = predict_from_frame(net, frame, model_utk, display_probs = True)

    parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name

    cv2.imwrite(os.path.join(parent, f_name+'_predicted.jpg'), frame)
    cv2_imshow(frame)
    # cv2_imshow('Face Detector', frame)
    key = cv2.waitKey(0) & 0x00
    if key == ord("q"):
      cv2.destroyAllWindows()


### PSE-Male


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
import pandas as pd

# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam/resnet_checkpoint_val_loss=-4.02144347319082.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            # do age, gender and race prediction
            # face = frame[startY-25: endY+25, startX-25: endX+25]
            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame, race_text
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])




    image_list = []
    res_list=[]
    fname_list=[]
    key_list = []
    res_list1=[]
    fname_list1=[]
    key_list1 = []

    for v in range(10):

      # image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)

      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m/M'+ str(v)

      for i in range(11):
        images = image_id+ '_'+ str(i) + '.jpg'

        source_file = images



        frame = cv2.imread(source_file)
        frame,race_text = predict_from_frame(net, frame, model_utk, display_probs = True)

        parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name

  

        cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010M/', f_name+'_predicted.jpg'), frame)
        

        cv2_imshow(frame)
        # cv2_imshow('Face Detector', frame)
        key = cv2.waitKey(0) & 0x00
        if key == ord("q"):
          cv2.destroyAllWindows()


######################################################################
        if race_text[2:7] == 'White':
          race_digit= 0
        else:
          race_digit= 1
        res_list.append(race_digit)

        fname_list.append(f_name)

        temp = f_name.strip('.jpg')
        temp = int(temp[3:])*10
        print(str(temp))
        key_list.append(temp)
        

        df = pd.DataFrame(
                {'id': ['9010M']*len(fname_list),
                'stim.name': fname_list,
                'choice': res_list})
        
        df.append(df)

        df_key = pd.DataFrame(
                {'stim.name': fname_list,
                'Racial.Percentage': key_list}) 

        df_key.append(df_key)
        print(df_key)

        df.to_csv('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010M/m_df.csv',  encoding='utf-8')
        df_key.to_csv('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010M/m_key.csv',  encoding='utf-8')




### mask - male


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam/resnet_checkpoint_val_loss=-4.02144347319082.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            # do age, gender and race prediction
            # face = frame[startY-25: endY+25, startX-25: endX+25]
            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])






    for v in range(11):
      image_list = []

      # image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)

      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m/M'+ str(v)

      for i in range(11):
        images = image_id+ '_'+ str(i) + '.jpg'

        source_file = images



        frame = cv2.imread(source_file)
        frame = predict_from_frame(net, frame, model_utk, display_probs = True)


        parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name
        cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010M/', f_name+'_predicted.jpg'), frame)
        
        cv2_imshow(frame)
        # cv2_imshow('Face Detector', frame)
        key = cv2.waitKey(0) & 0x00
        if key == ord("q"):
          cv2.destroyAllWindows()


### mask-female


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/9010/resnet_adam/resnet_checkpoint_val_loss=-4.02144347319082.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1]],
                   target_names=[['Male', 'Female'], ['White', 'Black']],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black']
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred



def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            # do age, gender and race prediction
            # face = frame[startY-25: endY+25, startX-25: endX+25]
            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

                race_text = f"{race_lab, round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame, race_text
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])




    image_list = []
    res_list=[]
    fname_list=[]
    key_list = []
    res_list1=[]
    fname_list1=[]
    key_list1 = []

    for v in range(10):
      image_list = []

      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)

      # image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m/M'+ str(v)

      for i in range(11):
        images = image_id+ '_'+ str(i) + '.jpg'

        source_file = images

        frame = cv2.imread(source_file)
        frame,race_text = predict_from_frame(net, frame, model_utk, display_probs = True)


        parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name
        cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010F/', f_name+'_predicted.jpg'), frame)
        
        cv2_imshow(frame)
        # cv2_imshow('Face Detector', frame)
        key = cv2.waitKey(0) & 0x00
        if key == ord("q"):
          cv2.destroyAllWindows()
        

        ######################################################################
        if race_text[2:7] == 'White':
          race_digit= 0
        else:
          race_digit= 1
        res_list.append(race_digit)

        fname_list.append(f_name)

        temp = f_name.strip('.jpg')
        temp = int(temp[3:])*10
        print(str(temp))
        key_list.append(temp)

        df = pd.DataFrame(
               {'id': ['9010M']*len(fname_list),
                'stim.name': fname_list,
                'choice': res_list})
        


        df_key = pd.DataFrame(
               {'stim.name': fname_list,
                'Racial.Percentage': key_list})    
        print(df_key)

        df.to_csv('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010F/f_df.csv',  encoding='utf-8')
        df_key.to_csv('/content/gdrive/My Drive/DeepLearning/Face_detection/test/9010F/f_key.csv',  encoding='utf-8')





### B/W VERSION - male


In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred


def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 3)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

######b/w version             
                if race_proba[0] > race_proba[1]:
                  white_prob = round(race_proba[0],3)
                  race_text = f"{'White', white_prob}"
                else: 
                  race_lab = race_labels[1]                  
                  black_prob = round(race_proba[1],3)
                  race_text = f"{'Black', black_prob}"
######b/w version
                # race_text = f"{'test', round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)
    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])



    for v in range(11):
      image_list = []

      # image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)

      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m/M'+ str(v)


      for i in range(11):
        images = image_id+ '_'+ str(i) + '.jpg'
        
          
        source_file = images


        frame = cv2.imread(source_file)
        frame = predict_from_frame(net, frame, model_utk, display_probs = True)


        parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name

        cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_m_bw_res/', f_name+'_predicted.jpg'), frame)
        
        cv2_imshow(frame)
        key = cv2.waitKey(0) & 0x00
        if key == ord("q"):
          cv2.destroyAllWindows()



### B/W Version - female

In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred


def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 1)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

######b/w version             
                if race_proba[0] > race_proba[1]:
                  white_prob = round(race_proba[0],3)
                  race_text = f"{'White', white_prob}"
                else: 
                  race_lab = race_labels[1]                  
                  black_prob = round(race_proba[1],3)
                  race_text = f"{'Black', black_prob}"
######b/w version
                # race_text = f"{'test', round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)


class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)


    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])




    for v in range(10):
      image_list = []

      image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)



      for i in range(11):
        images = image_id+ '_'+ str(i) + '.jpg'
        
          
        source_file = images

        frame = cv2.imread(source_file)
        frame = predict_from_frame(net, frame, model_utk, display_probs = True)


        parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name


        cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f_bw_res/', f_name+'_predicted.jpg'), frame)
        
        cv2_imshow(frame)
        # cv2_imshow('Face Detector', frame)
        key = cv2.waitKey(0) & 0x00
        if key == ord("q"):
          cv2.destroyAllWindows()



## GRAD-CAM

### warning: this is a mess. Ignore this section for now

In [ ]:

import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
from torch.autograd import Function
from torchvision import models
import json


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred


def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 1)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

######b/w version             
                if race_proba[0] > race_proba[1]:
                  white_prob = round(race_proba[0],3)
                  race_text = f"{'White', white_prob}"
                else: 
                  race_lab = race_labels[1]                  
                  black_prob = round(race_proba[1],3)
                  race_text = f"{'Black', black_prob}"
######b/w version
                # race_text = f"{'test', round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)

##########grad-cam starts

# class FeatureExtractor():
#     """ Class for extracting activations and 
#     registering gradients from targetted intermediate layers """

#     def __init__(self, model, target_layers):
#         self.model = model
#         self.target_layers = target_layers
#         self.gradients = []

#     def save_gradient(self, grad):
#         self.gradients.append(grad)

#     def __call__(self, x):
#         outputs = []
#         self.gradients = []
#         for name, module in self.model._modules.items():
#             x = module(x)
#             if name in self.target_layers:
#                 x.register_hook(self.save_gradient)
#                 outputs += [x]
#         return outputs, x


# class ModelOutputs():
#     """ Class for making a forward pass, and getting:
#     1. The network output.
#     2. Activations from intermeddiate targetted layers.
#     3. Gradients from intermeddiate targetted layers. """

#     def __init__(self, model, feature_module, target_layers):
#         self.model = model
#         self.feature_module = feature_module
#         self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

#     def get_gradients(self):
#         return self.feature_extractor.gradients

#     def __call__(self, x):
#         target_activations = []
#         for name, module in self.model._modules.items():
#             if module == self.feature_module:
#                 target_activations, x = self.feature_extractor(x)
#             elif "avgpool" in name.lower():
#                 x = module(x)
#                 x = x.view(x.size(0),-1)
#             else:
#                 x = module(x)
        
#         return target_activations, x


# def preprocess_image(img):
#     means = [0.485, 0.456, 0.406]
#     stds = [0.229, 0.224, 0.225]

#     preprocessed_img = img.copy()[:, :, ::-1]
#     for i in range(3):
#         preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
#         preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
#     preprocessed_img = \
#         np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
#     preprocessed_img = torch.from_numpy(preprocessed_img)
#     preprocessed_img.unsqueeze_(0)
#     input = preprocessed_img.requires_grad_(True)
#     return input


# def show_cam_on_image(img, mask):
#     heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
#     heatmap = np.float32(heatmap) / 255
#     cam = heatmap + np.float32(img)
#     cam = cam / np.max(cam)
#     cv2.imwrite("cam.jpg", np.uint8(255 * cam))


# class GradCam:
#     def __init__(self, model, feature_module, target_layer_names, use_cuda):
#         self.model = model
#         self.feature_module = feature_module
#         self.model.eval()
#         self.cuda = use_cuda
#         if self.cuda:
#             self.model = model.cuda()

#         self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

#     def forward(self, input):
#         return self.model(input)

#     def __call__(self, input, index=None):
#         if self.cuda:
#             features, output = self.extractor(input.cuda())
#         else:
#             features, output = self.extractor(input)

#         if index == None:
#             index = np.argmax(output.cpu().data.numpy())

#         one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
#         one_hot[0][index] = 1
#         one_hot = torch.from_numpy(one_hot).requires_grad_(True)
#         if self.cuda:
#             one_hot = torch.sum(one_hot.cuda() * output)
#         else:
#             one_hot = torch.sum(one_hot * output)

#         self.feature_module.zero_grad()
#         self.model.zero_grad()
#         one_hot.backward(retain_graph=True)

#         grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

#         target = features[-1]
#         target = target.cpu().data.numpy()[0, :]

#         weights = np.mean(grads_val, axis=(2, 3))[0, :]
#         cam = np.zeros(target.shape[1:], dtype=np.float32)

#         for i, w in enumerate(weights):
#             cam += w * target[i, :, :]

#         cam = np.maximum(cam, 0)
#         cam = cv2.resize(cam, input.shape[2:])
#         cam = cam - np.min(cam)
#         cam = cam / np.max(cam)
#         return cam


# class GuidedBackpropReLU(Function):

#     @staticmethod
#     def forward(self, input):
#         positive_mask = (input > 0).type_as(input)
#         output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
#         self.save_for_backward(input, output)
#         return output

#     @staticmethod
#     def backward(self, grad_output):
#         input, output = self.saved_tensors
#         grad_input = None

#         positive_mask_1 = (input > 0).type_as(grad_output)
#         positive_mask_2 = (grad_output > 0).type_as(grad_output)
#         grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input),
#                                    torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output,
#                                                  positive_mask_1), positive_mask_2)

#         return grad_input


# class GuidedBackpropReLUModel:
#     def __init__(self, model, use_cuda):
#         self.model = model
#         self.model.eval()
#         self.cuda = use_cuda
#         if self.cuda:
#             self.model = model.cuda()

#         def recursive_relu_apply(module_top):
#             for idx, module in module_top._modules.items():
#                 recursive_relu_apply(module)
#                 if module.__class__.__name__ == 'ReLU':
#                     module_top._modules[idx] = GuidedBackpropReLU.apply
                
#         # replace ReLU with GuidedBackpropReLU
#         recursive_relu_apply(self.model)

#     def forward(self, input):
#         return self.model(input)

#     def __call__(self, input, index=None):
#         if self.cuda:
#             output = self.forward(input.cuda())
#         else:
#             output = self.forward(input)

#         if index == None:
#             index = np.argmax(output.cpu().data.numpy())

#         one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
#         one_hot[0][index] = 1
#         one_hot = torch.from_numpy(one_hot).requires_grad_(True)
#         if self.cuda:
#             one_hot = torch.sum(one_hot.cuda() * output)
#         else:
#             one_hot = torch.sum(one_hot * output)

#         # self.model.features.zero_grad()
#         # self.model.classifier.zero_grad()
#         one_hot.backward(retain_graph=True)

#         output = input.grad.cpu().data.numpy()
#         output = output[0, :, :, :]

#         return output


# def get_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--use-cuda', action='store_true', default=True,
#                         help='Use NVIDIA GPU acceleration')
#     parser.add_argument('--image-path', type=str, default='./examples/both.png',
#                         help='Input image path')
#     args = parser.parse_args()
#     args.use_cuda = args.use_cuda and torch.cuda.is_available()
#     if args.use_cuda:
#         print("Using GPU for acceleration")
#     else:
#         print("Using CPU for computation")

#     return args

# def deprocess_image(img):
#     """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
#     img = img - np.mean(img)
#     img = img / (np.std(img) + 1e-5)
#     img = img * 0.1
#     img = img + 0.5
#     img = np.clip(img, 0, 1)
#     return np.uint8(img*255)
###########grad-cam ends



###########grad-cam starts


class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, pre_features, features, target_layers):
        self.model = model
        self.pre_features = pre_features
        self.features = features
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        
        for pref in self.pre_features:
            x = getattr(self.model, pref)(x)

        submodel = getattr(self.model, self.features)
        # go through the feature extractor's forward pass
        for name, module in submodel._modules.items():
            # print(name, module)
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model,
                 pre_feature_block=[],
                 feature_block='features',
                 target_layers='35',
                 classifier_block=['classifier']):
        self.model = model
        self.classifier_block = classifier_block
        # assume the model has a module named `feature`      ⬇⬇⬇⬇⬇⬇⬇⬇
        self.feature_extractor = FeatureExtractor(self.model,
                                                  pre_feature_block,
                                                  feature_block,
                                                  target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        # ⬇ target layer    ⬇ final layer's output
        target_activations, output = self.feature_extractor(x)
        print('target_activations[0].size: {}'.format(target_activations[0].size()))# for vgg'35 ([1, 512, 14, 14])
        print('output.size: {}'.format(output.size()))                              # for vgg'36 ([1, 512, 7, 7])

        for i, classifier in enumerate(self.classifier_block):
            if i == len(self.classifier_block) - 1:
                output = output.view(output.size(0), -1)
                print('output.view.size: {}'.format(output.size()))                 # for vgg'36 ([1, 25088])

            output = getattr(self.model, classifier)(output)
            print('output.size: {}'.format(output.size()))                          # for vgg'36 ([1, 1000])

        return target_activations, output


def preprocess_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0)
    input = preprocessed_img.requires_grad_(True)
    return input


def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    cv2.imwrite("cam.jpg", np.uint8(255 * cam))


class GradCam:
    def __init__(self, model, pre_feature_block, feature_block, target_layer_names, classifier_block, use_cuda):
        self.model = model
        self.model.eval()
        self.pre_feature_block = pre_feature_block
        self.feature_block = feature_block
        self.classifier_block = classifier_block
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model,
                                      pre_feature_block,
                                      feature_block,
                                      target_layer_names,
                                      classifier_block)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())

        with open('/content/gdrive/My Drive/DeepLearning/Face_detection/imagenet_class_index.json') as f:
            labels = json.load(f)
        
        print('prediction[{}]: {}'.format(index, labels[str(index)][1]))
        print('output.size: {}'.format(output.size()))                            # for vgg'36 ([1, 1000])
        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        #print('output: {}'.format(output))
        #print('one_hot: {}'.format(one_hot))                                      # 
        getattr(self.model, self.feature_block).zero_grad()
        for classifier in self.classifier_block:
            getattr(self.model, classifier).zero_grad()
        one_hot.backward(retain_graph=True)

        gradients = self.extractor.get_gradients()
        #print('len(gradients): {}'.format(len(gradients)))
        print('gradients[0].size(): {}'.format(gradients[0].size()))
        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        print('target.size(): {}'.format(target.size()))
        target = target.cpu().data.numpy()[0, :]
        print('target.shape: {}'.format(target.shape))

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        print('weights.shape: {}'.format(weights.shape))
        cam = np.zeros(target.shape[1:], dtype=np.float32)
        print('cam.shape: {}'.format(cam.shape))            # (14, 14)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        
        #print('cam: {}'.format(cam))
        print('cam.shape: {}'.format(cam.shape))
        cam = np.maximum(cam, 0)                            # remove negative numbers
        cam = cv2.resize(cam, (224, 224))
        print('cam.shape: {}'.format(cam.shape))
        #print('cam: {}'.format(cam))
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam


class GuidedBackpropReLU(Function):

    @staticmethod
    def forward(self, input):
        positive_mask = (input > 0).type_as(input)
        output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
        self.save_for_backward(input, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input),
                                   torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output,
                                                 positive_mask_1), positive_mask_2)

        return grad_input


class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        # replace ReLU with GuidedBackpropReLU
        for idx, module in self.model.features._modules.items():
            if module.__class__.__name__ == 'ReLU':
                self.model.features._modules[idx] = GuidedBackpropReLU.apply

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            output = self.forward(input.cuda())
        else:
            output = self.forward(input)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        one_hot.backward(retain_graph=True)

        output = input.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument('--image-path', type=str, default='./examples/both.png',
                        help='Input image path')
    args = parser.parse_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print("Using GPU for acceleration")
    else:
        print("Using CPU for computation")

    return args

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

###########grad-cam ends





class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    # model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)

    model_utk = PretrainedMT1('resnet', feature_extract=True, use_pretrained=True)

    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])




    # for v in range(10):
    #   image_list = []

    #   image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)



    #   for i in range(11):
    #     images = image_id+ '_'+ str(i) + '.jpg'
        
          
    #     source_file = images

    #     frame = cv2.imread(source_file)
    #     frame = predict_from_frame(net, frame, model_utk, display_probs = True)


    #     parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name


    #     cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f_bw_res/', f_name+'_predicted.jpg'), frame)
        



    #     cv2_imshow(frame)
    #     # cv2_imshow('Face Detector', frame)
    #     key = cv2.waitKey(0) & 0x00
    #     if key == ord("q"):
    #       cv2.destroyAllWindows()



    # model = model_utk
    # model = models.resnet50(pretrained=True)
    # grad_cam = GradCam(model=model, feature_module=model.layer4, \
    #                    target_layer_names=["2"], use_cuda="cuda" if torch.cuda.is_available() else "cpu")

    # img = cv2.imread("/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg", 1)
    # img = np.float32(cv2.resize(img, (224, 224))) / 255
    # input = preprocess_image(img)

    # # If None, returns the map for the highest scoring category.
    # # Otherwise, targets the requested index.
    # target_index = None
    # mask = grad_cam(input, target_index)

    # show_cam_on_image(img, mask)

    # gb_model = GuidedBackpropReLUModel(model=model, use_cuda= "cuda" if torch.cuda.is_available() else "cpu")
    # print(model._modules.items())
    # gb = gb_model(input, index=target_index)
    # gb = gb.transpose((1, 2, 0))
    # cam_mask = cv2.merge([mask, mask, mask])
    # cam_gb = deprocess_image(cam_mask*gb)
    # gb = deprocess_image(gb)

    # cv2.imwrite('gb.jpg', gb)
    # cv2.imwrite('cam_gb.jpg', cam_gb)
    # cv2_imshow(cam_gb) 
    # cv2_imshow(gb) 




    image_path = "/content/gdrive/My Drive/DeepLearning/Face_detection/test/yutsuru.jpg"
    use_cuda = True

    # Can work with any model, but it assumes that the model has a
    # feature method, and a classifier method,
    # as in the VGG models in torchvision.
    config = {
        'vgg19':    {
            'pre_feature': [],
            'features': 'features',
            'target': ['35'],
            'classifier': ['classifier']
        }, 
        'resnet50': {
            'pre_feature': ['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3'],
            'features': 'layer4',
            'target': ['2'],
            'classifier': ['avgpool', 'fc']
        }
    }

    model_name = 'resnet50'
    config = config[model_name]
    model = getattr(models, model_name)(pretrained=True)
    grad_cam = GradCam(model,
                       pre_feature_block=config['pre_feature'],
                       feature_block=config['features'], # features
                       target_layer_names=config['target'],
                       classifier_block=config['classifier'],  # classifier
                       use_cuda=use_cuda)

    img = cv2.imread(image_path, 1)
    img = np.float32(cv2.resize(img, (224, 224))) / 255

    #print('img.size(): {}'.format(img.size))
    input = preprocess_image(img)
    #print('input.size(): {}'.format(input.size()))

    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested index.
    target_index = None
    mask = grad_cam(input, target_index)
    
    show_cam_on_image(img, mask)


    # gb_model = GuidedBackpropReLUModel(model=model_utk, use_cuda=use_cuda)
    # gb = gb_model(input, index=target_index)
    # gb = gb.transpose((1, 2, 0))
    # cam_mask = cv2.merge([mask, mask, mask])
    # cam_gb = deprocess_image(cam_mask*gb)
    # gb = deprocess_image(gb)
    # cv2.imwrite('gb.jpg', gb)
    # cv2.imwrite('cam_gb.jpg', cam_gb)

    # cv2_imshow(cam_gb) 
    # cv2_imshow(gb) 

    '''
    gb_model = GuidedBackpropReLUModel(model=models.vgg19(pretrained=True), use_cuda=use_cuda)
    gb = gb_model(input, index=target_index)
    gb = gb.transpose((1, 2, 0))
    cam_mask = cv2.merge([mask, mask, mask])
    cam_gb = deprocess_image(cam_mask*gb)
    gb = deprocess_image(gb)
    cv2.imwrite('gb.jpg', gb)
    cv2.imwrite('cam_gb.jpg', cam_gb)
    '''

In [ ]:
import imutils
import cv2
import torch
from vision_utils.custom_architectures import PretrainedMT, SepConvModel, SepConvModelMT
from vision_utils.custom_torch_utils import initialize_model
from emotion_detection.evaluate import predict_fer
from multitask_rag.evaluate import predict_utk
import numpy as np
import os
import pathlib
import argparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
from torch.autograd import Function
from torchvision import models
import json


# default path to a saved model for race, age and gender prediction
# saved_weight_utk = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'multitask_rag/checkpoints/vgg_model_21_val_loss=4.139335.pth'


# saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.217711824612125.pth"
saved_weight_utk ="/content/gdrive/My Drive/DeepLearning/Face_detection/checkpoints/resnet_adam/resnet_checkpoint_val_loss=-6.249345162580937.pth"

# default path to a saved model for emotion prediction
# saved_weight_fer = '/media/sf_Documents/COMPUTER_VISION/DEmoClassi/' \
#                    'emotion_detection/checkpoints/vgg_model_173_val_accuracy=0.6447478.pth'


# paths to the caffe model files for detecting faces using opencv
# package_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
package_path = os.path.dirname(os.path.dirname(os.path.abspath('/content/gdrive/My Drive/DEmoClassi-master/cv2_dnn_model_files')))
path_binaries = os.path.join(package_path, 'cv2_dnn_model_files')
path_caffe_model = os.path.join(path_binaries, 'res10_300x300_ssd_iter_140000.caffemodel')
path_proto = os.path.join(path_binaries, 'deploy.prototxt.txt')

def dict_prob_to_list(dict_probs):
    """
    utility function for converting a dictionary of labels with their probabilities
     into two lists of labels and probs resp.
    """
    items = list(dict_probs.items())
    return [item[0] for item in items], [item[1] for item in items]


def plot_to_array(x, y, color):
    """Utility function for ploting predicted probabilities as bar plots"""
    fig = plt.figure(figsize=(2, 2))
    fig.add_subplot(111)
    # fig.tight_layout(pad=0)
    plt.barh(x, y, color=color)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return data

def evaluate_model(model, dataloader,
                   title='Confusion matrix',
                   labels_=[[0, 1], [0, 1, 2, 3, 4, 5,6 ]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix
        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for inputs, age, gender, race in tqdm.tqdm(dataloader):
            inputs = inputs.to(device, dtype=torch.float32)
            y_age.append(age)
            y_gender.append(gender)
            y_race.append(race)

            age_pred, gender_pred, race_pred = model(inputs)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())

    # print classification report
    y_age, y_pred_age = np.concatenate(y_age), np.concatenate(y_pred_age)
    y_gender, y_pred_gender = np.concatenate(y_gender), np.concatenate(y_pred_gender)
    y_race, y_pred_race = np.concatenate(y_race), np.concatenate(y_pred_race)

    print('----------------------- Age prediction -------------------------')
    print(f"Mean Absolute Error {np.abs(y_age - y_pred_age).mean():.4f}")

    print('----------------------- Gender prediction -------------------------')
    plot_confusion_matrix(y_gender, y_pred_gender, title, labels_[0], target_names[0], normalize)

    print('----------------------- Race prediction -------------------------')
    plot_confusion_matrix(y_race, y_pred_race, title, labels_[1], target_names[1], normalize)


def preprocess_utk(image):
    transf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    return transf(image).unsqueeze_(0)


def predict_utk(image, model):

    # process image
    image = preprocess_utk(image)

    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    age_pred, gender_pred, race_pred = model(image)
    age_pred = age_pred.detach().to('cpu').numpy()[0][0]
    gender_probs, race_probs = F.softmax(gender_pred, dim=1).detach().to('cpu').numpy()[0],\
                               F.softmax(race_pred, dim=1).detach().to('cpu').numpy()[0]

    # map probabilities to label names
    gender_labs, race_labs = ['Male', 'Female'], ['White', 'Black', 'East Asian', 'Southeast Asian', 'Middle Eastern','Latino_Hispanic', 'Indian' ]
    gender_label_pred = gender_labs[np.argmax(gender_probs)]
    race_label_pred = race_labs[np.argmax(race_probs)]

    gender = dict(zip(gender_labs, gender_probs))
    race = dict(zip(race_labs, race_probs))

    return age_pred, gender, gender_label_pred, race, race_label_pred


def predict_from_frame(net, frame, model_utk, display_probs):
    """
    Makes emotion, gender, age and race prediction from a frame and plot the results in the frame to display
     using opencv
    :param net: opencv face detector
    :param frame: numpy array representing the image from hich to detect face and make prediction
    :param model_utk: pytorch model for predicting race, age and gender
    # :param model_fer: pytorch model for predicting emotion
    #:param transfer_learn: whether we are using a pretrained model (`resnet` or `vgg`)
    :param display_probs: True or False, whether to plot the predicted probabilities for each class
    :return:
    """
    # transfer_learn = True
    frame = imutils.resize(frame, width=600, height=600)

    # Prepare the opencv face detector
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # iterate through the detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # if the model has detected face with at least 50% confidence
        # get the bounding box of the face and plot it
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX - 25, startY - 50), (endX + 25, endY + 25), (0, 255, 0), 1)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 2)

            face = frame[startY: endY, startX: endX]
            age, gender, gender_lab, race, race_lab = predict_utk(face, model_utk)

  
            try:
                if race_lab == "Latino_Hispanic":
                  race_lab = "L/H"
                if race_lab == "East Asian":
                  race_lab = "E Asian"
                if race_lab == "Southeast Asian":
                  race_lab = "SE Asian"
                if race_lab == "Middle Eastern":
                  race_lab = "ME"


                if gender_lab == "Male":
                  gender_lab = "M"
                if gender_lab == "Female":
                  gender_lab = "F"

                gender_labels, gender_proba = dict_prob_to_list(gender)
                race_labels,  race_proba = dict_prob_to_list(race)

######b/w version             
                if race_proba[0] > race_proba[1]:
                  white_prob = round(race_proba[0],3)
                  race_text = f"{'White', white_prob}"
                else: 
                  race_lab = race_labels[1]                  
                  black_prob = round(race_proba[1],3)
                  race_text = f"{'Black', black_prob}"
######b/w version
                # race_text = f"{'test', round(max(race_proba),3)}"
                gender_text = f"{gender_lab, round(max(gender_proba),3)}"
                print(race_text, gender_text)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, race_text, (startX , startY + 12), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, gender_text, (startX , endY - 2), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass
    return frame
    print('ojbk')
    print(text)

##########grad-cam starts

# class FeatureExtractor():
#     """ Class for extracting activations and 
#     registering gradients from targetted intermediate layers """

#     def __init__(self, model, target_layers):
#         self.model = model
#         self.target_layers = target_layers
#         self.gradients = []

#     def save_gradient(self, grad):
#         self.gradients.append(grad)

#     def __call__(self, x):
#         outputs = []
#         self.gradients = []
#         for name, module in self.model._modules.items():
#             x = module(x)
#             if name in self.target_layers:
#                 x.register_hook(self.save_gradient)
#                 outputs += [x]
#         return outputs, x


# class ModelOutputs():
#     """ Class for making a forward pass, and getting:
#     1. The network output.
#     2. Activations from intermeddiate targetted layers.
#     3. Gradients from intermeddiate targetted layers. """

#     def __init__(self, model, feature_module, target_layers):
#         self.model = model
#         self.feature_module = feature_module
#         self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

#     def get_gradients(self):
#         return self.feature_extractor.gradients

#     def __call__(self, x):
#         target_activations = []
#         for name, module in self.model._modules.items():
#             if module == self.feature_module:
#                 target_activations, x = self.feature_extractor(x)
#             elif "avgpool" in name.lower():
#                 x = module(x)
#                 x = x.view(x.size(0),-1)
#             else:
#                 x = module(x)
        
#         return target_activations, x


# def preprocess_image(img):
#     means = [0.485, 0.456, 0.406]
#     stds = [0.229, 0.224, 0.225]

#     preprocessed_img = img.copy()[:, :, ::-1]
#     for i in range(3):
#         preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
#         preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
#     preprocessed_img = \
#         np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
#     preprocessed_img = torch.from_numpy(preprocessed_img)
#     preprocessed_img.unsqueeze_(0)
#     input = preprocessed_img.requires_grad_(True)
#     return input


# def show_cam_on_image(img, mask):
#     heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
#     heatmap = np.float32(heatmap) / 255
#     cam = heatmap + np.float32(img)
#     cam = cam / np.max(cam)
#     cv2.imwrite("cam.jpg", np.uint8(255 * cam))


# class GradCam:
#     def __init__(self, model, feature_module, target_layer_names, use_cuda):
#         self.model = model
#         self.feature_module = feature_module
#         self.model.eval()
#         self.cuda = use_cuda
#         if self.cuda:
#             self.model = model.cuda()

#         self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

#     def forward(self, input):
#         return self.model(input)

#     def __call__(self, input, index=None):
#         if self.cuda:
#             features, output = self.extractor(input.cuda())
#         else:
#             features, output = self.extractor(input)

#         if index == None:
#             index = np.argmax(output.cpu().data.numpy())

#         one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
#         one_hot[0][index] = 1
#         one_hot = torch.from_numpy(one_hot).requires_grad_(True)
#         if self.cuda:
#             one_hot = torch.sum(one_hot.cuda() * output)
#         else:
#             one_hot = torch.sum(one_hot * output)

#         self.feature_module.zero_grad()
#         self.model.zero_grad()
#         one_hot.backward(retain_graph=True)

#         grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

#         target = features[-1]
#         target = target.cpu().data.numpy()[0, :]

#         weights = np.mean(grads_val, axis=(2, 3))[0, :]
#         cam = np.zeros(target.shape[1:], dtype=np.float32)

#         for i, w in enumerate(weights):
#             cam += w * target[i, :, :]

#         cam = np.maximum(cam, 0)
#         cam = cv2.resize(cam, input.shape[2:])
#         cam = cam - np.min(cam)
#         cam = cam / np.max(cam)
#         return cam


# class GuidedBackpropReLU(Function):

#     @staticmethod
#     def forward(self, input):
#         positive_mask = (input > 0).type_as(input)
#         output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
#         self.save_for_backward(input, output)
#         return output

#     @staticmethod
#     def backward(self, grad_output):
#         input, output = self.saved_tensors
#         grad_input = None

#         positive_mask_1 = (input > 0).type_as(grad_output)
#         positive_mask_2 = (grad_output > 0).type_as(grad_output)
#         grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input),
#                                    torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output,
#                                                  positive_mask_1), positive_mask_2)

#         return grad_input


# class GuidedBackpropReLUModel:
#     def __init__(self, model, use_cuda):
#         self.model = model
#         self.model.eval()
#         self.cuda = use_cuda
#         if self.cuda:
#             self.model = model.cuda()

#         def recursive_relu_apply(module_top):
#             for idx, module in module_top._modules.items():
#                 recursive_relu_apply(module)
#                 if module.__class__.__name__ == 'ReLU':
#                     module_top._modules[idx] = GuidedBackpropReLU.apply
                
#         # replace ReLU with GuidedBackpropReLU
#         recursive_relu_apply(self.model)

#     def forward(self, input):
#         return self.model(input)

#     def __call__(self, input, index=None):
#         if self.cuda:
#             output = self.forward(input.cuda())
#         else:
#             output = self.forward(input)

#         if index == None:
#             index = np.argmax(output.cpu().data.numpy())

#         one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
#         one_hot[0][index] = 1
#         one_hot = torch.from_numpy(one_hot).requires_grad_(True)
#         if self.cuda:
#             one_hot = torch.sum(one_hot.cuda() * output)
#         else:
#             one_hot = torch.sum(one_hot * output)

#         # self.model.features.zero_grad()
#         # self.model.classifier.zero_grad()
#         one_hot.backward(retain_graph=True)

#         output = input.grad.cpu().data.numpy()
#         output = output[0, :, :, :]

#         return output


# def get_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--use-cuda', action='store_true', default=True,
#                         help='Use NVIDIA GPU acceleration')
#     parser.add_argument('--image-path', type=str, default='./examples/both.png',
#                         help='Input image path')
#     args = parser.parse_args()
#     args.use_cuda = args.use_cuda and torch.cuda.is_available()
#     if args.use_cuda:
#         print("Using GPU for acceleration")
#     else:
#         print("Using CPU for computation")

#     return args

# def deprocess_image(img):
#     """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
#     img = img - np.mean(img)
#     img = img / (np.std(img) + 1e-5)
#     img = img * 0.1
#     img = img + 0.5
#     img = np.clip(img, 0, 1)
#     return np.uint8(img*255)
###########grad-cam ends



###########grad-cam starts

import torch
import matplotlib.pyplot as plt
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import sys
from collections import OrderedDict
import numpy as np
import argparse
import os
import torch.nn as nn
i=0##testing in what
resnet = models.resnet50(pretrained=True)#这里单独加载一个包含全连接层的resnet50模型
image = []
class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """
    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
    	self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():##resnet50没有.feature这个特征，直接删除用就可以。
            x = module(x)
            #print('name=',name)
            #print('x.size()=',x.size())
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
            #print('outputs.size()=',x.size())
        #print('len(outputs)',len(outputs))
        return outputs, x

class ModelOutputs():
	""" Class for making a forward pass, and getting:
	1. The network output.
	2. Activations from intermeddiate targetted layers.
	3. Gradients from intermeddiate targetted layers. """
	def __init__(self, model, target_layers,use_cuda):
		self.model = model
		self.feature_extractor = FeatureExtractor(self.model, target_layers)
		self.cuda = use_cuda
	def get_gradients(self):
		return self.feature_extractor.gradients

	def __call__(self, x):
		target_activations, output  = self.feature_extractor(x)
		output = output.view(output.size(0), -1)
		#print('classfier=',output.size())
		if self.cuda:
			output = output.cpu()
			output = resnet.fc(output).cuda()##这里就是为什么我们多加载一个resnet模型进来的原因，因为后面我们命名的model不包含fc层，但是这里又偏偏要使用。#
		else:
			output = resnet.fc(output)##这里对应use-cuda上更正一些bug,不然用use-cuda的时候会导致类型对不上,这样保证既可以在cpu上运行,gpu上运行也不会出问题.
		return target_activations, output

def preprocess_image(img):
	means=[0.485, 0.456, 0.406]
	stds=[0.229, 0.224, 0.225]

	preprocessed_img = img.copy()[: , :, ::-1]
	for i in range(3):
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
	preprocessed_img = \
		np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
	preprocessed_img = torch.from_numpy(preprocessed_img)
	preprocessed_img.unsqueeze_(0)
	input = preprocessed_img
	input.requires_grad = True
	return input

def show_cam_on_image(img, mask,name):
	heatmap = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
	heatmap = np.float32(heatmap) / 255
	cam = heatmap + np.float32(img)
	cam = cam / np.max(cam)
	cv2.imwrite("cam/cam_{}.jpg".format(name), np.uint8(255 * cam))
class GradCam:
	def __init__(self, model, target_layer_names, use_cuda):
		self.model = model
		self.model.eval()
		self.cuda = use_cuda
		if self.cuda:
			self.model = model.cuda()

		self.extractor = ModelOutputs(self.model, target_layer_names, use_cuda)

	def forward(self, input):
		return self.model(input) 

	def __call__(self, input, index = None):
		if self.cuda:
			features, output = self.extractor(input.cuda())
		else:
			features, output = self.extractor(input)

		if index == None:
			index = np.argmax(output.cpu().data.numpy())

		one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
		one_hot[0][index] = 1
		one_hot = torch.Tensor(torch.from_numpy(one_hot))
		one_hot.requires_grad = True
		if self.cuda:
			one_hot = torch.sum(one_hot.cuda() * output)
		else:
			one_hot = torch.sum(one_hot * output)

		self.model.zero_grad()##features和classifier不包含，可以重新加回去试一试，会报错不包含这个对象。
		#self.model.zero_grad()
		one_hot.backward(retain_graph=True)##这里适配我们的torch0.4及以上，我用的1.0也可以完美兼容。（variable改成graph即可）

		grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()
		#print('grads_val',grads_val.shape)
		target = features[-1]
		target = target.cpu().data.numpy()[0, :]

		weights = np.mean(grads_val, axis = (2, 3))[0, :]
		#print('weights',weights.shape)
		cam = np.zeros(target.shape[1 : ], dtype = np.float32)
		#print('cam',cam.shape)
		#print('features',features[-1].shape)
		#print('target',target.shape)
		for i, w in enumerate(weights):
			cam += w * target[i, :, :]

		cam = np.maximum(cam, 0)
		cam = cv2.resize(cam, (224, 224))
		cam = cam - np.min(cam)
		cam = cam / np.max(cam)
		return cam
class GuidedBackpropReLUModel:
	def __init__(self, model, use_cuda):
		self.model = model#这里同理，要的是一个完整的网络，不然最后维度会不匹配。
		self.model.eval()
		self.cuda = use_cuda
		if self.cuda:
			self.model = model.cuda()
		for module in self.model.named_modules():
			module[1].register_backward_hook(self.bp_relu)

	def bp_relu(self, module, grad_in, grad_out):
		if isinstance(module, nn.ReLU):
			return (torch.clamp(grad_in[0], min=0.0),)
	def forward(self, input):
		return self.model(input)

	def __call__(self, input, index = None):
		if self.cuda:
			output = self.forward(input.cuda())
		else:
			output = self.forward(input)
		if index == None:
			index = np.argmax(output.cpu().data.numpy())
		#print(input.grad)
		one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
		one_hot[0][index] = 1
		one_hot = torch.from_numpy(one_hot)
		one_hot.requires_grad = True
		if self.cuda:
			one_hot = torch.sum(one_hot.cuda() * output)
		else:
			one_hot = torch.sum(one_hot * output)
		#self.model.classifier.zero_grad()
		one_hot.backward(retain_graph=True)
		output = input.grad.cpu().data.numpy()
		output = output[0,:,:,:]

		return output

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument('--use-cuda', action='store_true', default=False,
	                    help='Use NVIDIA GPU acceleration')
	parser.add_argument('--image-path', type=str, default='./examples/',
	                    help='Input image path')
	args = parser.parse_args()
	args.use_cuda = args.use_cuda and torch.cuda.is_available()
	if args.use_cuda:
	    print("Using GPU for acceleration")
	else:
	    print("Using CPU for computation")

	return args

###########grad-cam ends





class ConvModelMultiTask(nn.Module):
    """custom Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(ConvModelMultiTask, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, 'utk', use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race


    my_model = ConvModelMultiTask()
    # Define the optimizer
    optimizer = optim.Adam(
        [
            {"params": my_model.conv_base.fc.parameters(), "lr": 1e-3},
            {"params": my_model.output_age.parameters(), "lr": 1e-3},
            {"params": my_model.output_gender.parameters(), "lr": 1e-3},
            {"params": my_model.output_race.parameters(), "lr": 1e-3},
            {"params": my_model.conv_base.conv1.parameters()},
            {"params": my_model.conv_base.layer1.parameters()},
            {"params": my_model.conv_base.layer2.parameters()},
            {"params": my_model.conv_base.layer3.parameters()},
            {"params": my_model.conv_base.layer4.parameters()},
        ],
        lr=1e-6,
    )

class PretrainedMT1(nn.Module):
    """Pretrained Pytorch neural network module for multitask learning"""

    def __init__(self, model_name='resnet', feature_extract=True, use_pretrained=True):
        super(PretrainedMT1, self).__init__()
        self.conv_base, input_size = initialize_model(model_name, feature_extract, num_classes=None,
                                                      task='utk', use_pretrained=use_pretrained)
        self.output_age = nn.Linear(128, 1)
        self.output_gender = nn.Linear(128, 2)
        self.output_race = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv_base(x)
        age = self.output_age(x)
        gender = self.output_gender(x)
        race = self.output_race(x)
        return age, gender, race





if __name__ == '__main__':

    # opencv face detector
    cv2_facenet = cv2.dnn.readNetFromCaffe(path_proto, path_caffe_model)

    # start detection and prediction
    # main(predict_args, cv2_facenet)

    display_probs = True
    net = cv2_facenet     
    
    # model_utk = PretrainedMT1('resnet', feature_extract=False, use_pretrained=False)

    model_utk = PretrainedMT1('resnet', feature_extract=True, use_pretrained=True)

    model_utk.load_state_dict(torch.load(saved_weight_utk, map_location='cpu')['model'])




    # for v in range(10):
    #   image_list = []

    #   image_id = '/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f/F'+ str(v)



    #   for i in range(11):
    #     images = image_id+ '_'+ str(i) + '.jpg'
        
          
    #     source_file = images

    #     frame = cv2.imread(source_file)
    #     frame = predict_from_frame(net, frame, model_utk, display_probs = True)


    #     parent, f_name = str(pathlib.Path(source_file).parent), pathlib.Path(source_file).name


    #     cv2.imwrite(os.path.join('/content/gdrive/My Drive/DeepLearning/Face_detection/test/mask_f_bw_res/', f_name+'_predicted.jpg'), frame)
        



    #     cv2_imshow(frame)
    #     # cv2_imshow('Face Detector', frame)
    #     key = cv2.waitKey(0) & 0x00
    #     if key == ord("q"):
    #       cv2.destroyAllWindows()



    # model = model_utk
    # model = models.resnet50(pretrained=True)
    # grad_cam = GradCam(model=model, feature_module=model.layer4, \
    #                    target_layer_names=["2"], use_cuda="cuda" if torch.cuda.is_available() else "cpu")

    # img = cv2.imread("/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg", 1)
    # img = np.float32(cv2.resize(img, (224, 224))) / 255
    # input = preprocess_image(img)

    # # If None, returns the map for the highest scoring category.
    # # Otherwise, targets the requested index.
    # target_index = None
    # mask = grad_cam(input, target_index)

    # show_cam_on_image(img, mask)

    # gb_model = GuidedBackpropReLUModel(model=model, use_cuda= "cuda" if torch.cuda.is_available() else "cpu")
    # print(model._modules.items())
    # gb = gb_model(input, index=target_index)
    # gb = gb.transpose((1, 2, 0))
    # cam_mask = cv2.merge([mask, mask, mask])
    # cam_gb = deprocess_image(cam_mask*gb)
    # gb = deprocess_image(gb)

    # cv2.imwrite('gb.jpg', gb)
    # cv2.imwrite('cam_gb.jpg', cam_gb)
    # cv2_imshow(cam_gb) 
    # cv2_imshow(gb) 


    args = get_args()

    # Can work with any model, but it assumes that the model has a 
    # feature method, and a classifier method,
    # as in the VGG models in torchvision.
    model = models.resnet50(pretrained=True)#这里相对vgg19而言我们处理的不一样，这里需要删除fc层，因为后面model用到的时候会用不到fc层，只查到fc层之前的所有层数。
    del model.fc
    print(model)
    #modules = list(resnet.children())[:-1]
    #model = torch.nn.Sequential(*modules)



    image_path = "/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg"
    #print(model)
    grad_cam = GradCam(model , \
            target_layer_names = ["layer4"], use_cuda="cuda" if torch.cuda.is_available() else "cpu")##这里改成layer4也很简单，我把每层name和size都打印出来了，想看哪层自己直接嵌套就可以了。（最后你会在终端看得到name的）
    # x=os.walk(args.image_path)

    x=os.walk("/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg")
    for root,dirs,filename in x:
    #print(type(grad_cam))
      print(filename)
    for s in filename:

          # image.append(cv2.imread("/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg",1))
          # image.append(cv2.imread(args.image_path+s,1))
      #img = cv2.imread(filename, 1)
      img = cv2.imread("/content/gdrive/My Drive/DeepLearning/Face_detection/test/cui.jpg", 1)
      
    for img in image:
      img = np.float32(cv2.resize(img, (224, 224))) / 255
      input = preprocess_image(img)
      input.required_grad = True
      print('input.size()=',input.size())
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested index.
      target_index =None

      mask = grad_cam(input, target_index)
      i=i+1 
      show_cam_on_image(img, mask,i)

      gb_model = GuidedBackpropReLUModel(model = models.resnet50(pretrained=True), use_cuda="cuda" if torch.cuda.is_available() else "cpu")
      gb = gb_model(input, index=target_index)
      if not os.path.exists('gb'):
        os.mkdir('gb')
      if not os.path.exists('camgb'):
        os.mkdir('camgb')
      utils.save_image(torch.from_numpy(gb), 'gb/gb_{}.jpg'.format(i))
      cam_mask = np.zeros(gb.shape)
      for j in range(0, gb.shape[0]):
        cam_mask[j, :, :] = mask
      cam_gb = np.multiply(cam_mask, gb)
      utils.save_image(torch.from_numpy(cam_gb), 'camgb/cam_gb_{}.jpg'.format(i))

???

In [ ]:
import argparse
import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models

class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)
        
        return target_activations, x


def preprocess_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0)
    input = preprocessed_img.requires_grad_(True)
    return input


def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    cv2.imwrite("cam.jpg", np.uint8(255 * cam))


class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        target = target.cpu().data.numpy()[0, :]

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam


class GuidedBackpropReLU(Function):

    @staticmethod
    def forward(self, input):
        positive_mask = (input > 0).type_as(input)
        output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
        self.save_for_backward(input, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input),
                                   torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output,
                                                 positive_mask_1), positive_mask_2)

        return grad_input


class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply
                
        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            output = self.forward(input.cuda())
        else:
            output = self.forward(input)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        # self.model.features.zero_grad()
        # self.model.classifier.zero_grad()
        one_hot.backward(retain_graph=True)

        output = input.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=True,
                        help='Use NVIDIA GPU acceleration')
    # parser.add_argument('--image-path', type=str, default='./examples/both.png',
    #                     help='Input image path')
    args = parser.parse_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print("Using GPU for acceleration")
    else:
        print("Using CPU for computation")

    return args

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)


if __name__ == '__main__':
    """ python grad_cam.py <path_to_image>
    1. Loads an image with opencv.
    2. Preprocesses it for VGG19 and converts to a pytorch variable.
    3. Makes a forward pass to find the category index with the highest score,
    and computes intermediate activations.
    Makes the visualization. """

    args = get_args()

    # Can work with any model, but it assumes that the model has a
    # feature method, and a classifier method,
    # as in the VGG models in torchvision.

    model = models.resnet50(pretrained=True)
    grad_cam = GradCam(model=model, feature_module=model.layer4, \
                       target_layer_names=["2"], use_cuda=args.use_cuda)

    img = cv2.imread("/content/gdrive/My Drive/DeepLearning/Face_detection/test/test25.jpg", 1)
    img = np.float32(cv2.resize(img, (224, 224))) / 255
    input = preprocess_image(img)

    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested index.
    target_index = None
    mask = grad_cam(input, target_index)

    show_cam_on_image(img, mask)

    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=args.use_cuda)
    print(model._modules.items())
    gb = gb_model(input, index=target_index)
    gb = gb.transpose((1, 2, 0))
    cam_mask = cv2.merge([mask, mask, mask])
    cam_gb = deprocess_image(cam_mask*gb)
    gb = deprocess_image(gb)

    cv2.imwrite('gb.jpg', gb)
    cv2.imwrite('cam_gb.jpg', cam_gb)